<a href="https://colab.research.google.com/github/jazib/ChatApp/blob/master/curation_summary/tests/10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install requirements


In [51]:
!pip install langchain
!pip install openai
# !pip install anthropic

In [ ]:
from pydantic.main import BaseModel, Field
from langchain.chat_models import ChatOpenAI, ChatAnthropic
from langchain.output_parsers import ResponseSchema, StructuredOutputParser, PydanticOutputParser
from langchain.callbacks import get_openai_callback
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser
import pandas as pd
from langchain.schema import HumanMessage, AIMessage

### Set up LLMs

In [ ]:
# Set Open AI Key
import os
import getpass
OPENAI_API_KEY = getpass.getpass("Enter OpenAI API Key")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


Enter OpenAI API Key··········


In [ ]:
# Set Anthropic AI Key
# import os
# import getpass
# ANTHROPIC_API_KEY = getpass.getpass("Enter Anthropic API Key")
# os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

In [ ]:
# Initialize chat via OpenAI
chat = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")

# Initialize chat via Anthropic
# chat = ChatAnthropic(temperature=0.0)

### Set up prompts

In [29]:
# Set up prompt templates to pass to the LLM as a conversation
system_template = """\
You are a tech recruiter at a Silicon Valley technology startup. You are reviewing software developers for their fit at your company.

To assess whether a candidate should proceed or be rejected, use every one of the following assessment steps delimited by triple backticks below:

```
Evaluate a candidate based on the following criteria:

1. You only hire candidates who have work experience directly related to software application development. Specifically, you want candidates who could work in the following areas: [Frontend, Backend, Full Stack, DevOps, QA, Mobile, Data Science, Data Engineering, Data Analyst, or Engineering Lead]. If their most recent work history falls outside software development, reject them with the code "NOT_CORE_ROLE".

2. Infer which country the candidate comes from. In order to proceed in hiring, the candidate must come from one of the following approved countries [Canada, Mexico, Colombia, Chile, Costa Rica, Poland, Spain, Romania, Hungary]. You reject candidates from any other country. If the candidate is not from the right country, use the rejection code "OUTSIDE_GEO".

3. Infer the total years of experience the candidate has as a software developer. Calculate this by surveying all the items in their candidate_work_experiences and using just the jobs related to software engineering. Use the job's start_date in YYYY-MM-DD format to understand when they began each job. Many end_dates are missing, especially for current jobs, so be aware today's data is July 31, 2023. In order to proceed in hiring, the candidate must have 2 or more years of total hands-on software development experience. If the candidate does not have at least 2 years of experience, use the rejection code "TOO_JUNIOR".

4. Infer whether the candidate has Tech/Team Lead experience. You prefer candidates with tech or team lead experience over those without, but this is not a requirement just a nice-to-have.

5. Infer whether the candidate is an individual contributor or a manager. If the candidate is a senior manager and has been focused on management or executive roles for more than 3 years, use the rejection code "NOT_IC".

6. Infer the candidate's technology skills from both their set of skills and their job titles. In order for the candidate to proceed, they should work with modern tools and frameworks. If the candidate only works with old tools like C and Java, we are not interested in them and they should be rejected with the code "NOT_CORE_FOCUS".

7. You should never be analyzing communication skills for this candidate, that is not something to consider at this time.
```

A human will share a candidate with you to evaluate. Before you respond, fully evaluate the candidate using the criteria above. Come up with your answers for every step in order to create an entire mental image of this candidate from which you can evaluate the candidate as a whole.

Proceed in the following way:

```
1. Consider the candidate as a whole. How many years of experience do they have as a software developer? What has their career progression looked like? Have they leveled up quickly or slowly? Have they mostly worked as a developer or in other jobs?
2. Decide if you think we should proceed with them in the hiring process or reject them.
2. Share your decision in a structured format based on the formatting instructions provided as follows.
```

{format_instructions}
"""

human_training_template_1 = """\
Analyze the following candidate:

```
{
  "id": 640056,
  "country": "Colombia",
  "candidate_skills": [
    {
      "skill": {
        "name": "Python"
      }
    },
    {
      "skill": {
        "name": "JavaScript"
      }
    },
    {
      "skill": {
        "name": "Postgres"
      }
    },
    {
      "skill": {
        "name": "AWS"
      }
    },
    {
      "skill": {
        "name": "Docker"
      }
    },
    {
      "skill": {
        "name": "NoSQL"
      }
    },
    {
      "skill": {
        "name": "Git"
      }
    },
    {
      "skill": {
        "name": "Linux"
      }
    },
    {
      "skill": {
        "name": "Golang"
      }
    },
    {
      "skill": {
        "name": "API"
      }
    },
    {
      "skill": {
        "name": "Postman"
      }
    },
    {
      "skill": {
        "name": "Microservices"
      }
    },
    {
      "skill": {
        "name": "Firebase"
      }
    },
    {
      "skill": {
        "name": "CI/CD"
      }
    }
  ],
  "candidate_work_experiences": [
    {
      "job_title": "Software Developer",
      "start_date": "2017-07-01",
      "end_date": "2022-08-01"
    },
    {
      "job_title": "Software Developer",
      "start_date": "2022-09-01",
      "end_date": null
    }
  ],
  "candidate_educations": [
    {
      "degree": "Bachelors",
      "school_name": "Icesi",
      "start_year": 2017,
      "graduation_year": 2022
    }
  ]
}
```
"""

human_training_template_2 = """\
Analyze the following candidate:

```
{
  "id": 636699,
  "country": "Colombia",
  "candidate_skills": [
    {
      "skill": {
        "name": "Python"
      }
    },
    {
      "skill": {
        "name": "AWS"
      }
    },
    {
      "skill": {
        "name": "Java"
      }
    },
    {
      "skill": {
        "name": "Docker"
      }
    },
    {
      "skill": {
        "name": "Scala"
      }
    },
    {
      "skill": {
        "name": "API"
      }
    }
  ],
  "candidate_work_experiences": [
    {
      "job_title": "Full Stack Developer III and Team Lead, GDX Travel ",
      "start_date": "2017-07-01",
      "end_date": "2019-12-01"
    },
    {
      "job_title": "Software Engineering manager",
      "start_date": "2021-08-01",
      "end_date": "2023-06-01"
    },
    {
      "job_title": "Software Development Team Lead",
      "start_date": "2019-12-01",
      "end_date": "2021-08-01"
    }
  ],
  "candidate_educations": [
    {
      "degree": "Bachelors",
      "school_name": "Universidad Central de Colombia",
      "start_year": 2010,
      "graduation_year": 2016
    }
  ],
  "candidate_curation_workflows": [
    {
      "rejected_reason": null
    }
  ]
}
```
"""

ai_training_response_template_1 = """\
```
{
  "pass_": "Yes",
  "rejection_reason": "N/A",
  "summary": "- 6 years of work experience as a software developer.\\n  - Bachelor\'s degree in a relevant field.\\n  - Experience with modern skills: Python, JavaScript, Postgres, AWS, Docker, and Git.\\n  - Worked as a software developer in their most recent roles.\\n  - No tech lead experience.\\n  - No information available about their level of responsibility or career progression.",
  "level": "IC3",
  "score_experience": 72,
  "score_tech_lead": false,
  "score_modern_tech": "Many",
  "score_country": true
}
```
"""


ai_training_response_template_2 = """\
```
{
  "pass_": "Yes",
  "rejection_reason": "N/A",
  "summary": "Bachelor\'s degree in software engineering from a reputable university.\n  - Total of 6 years of experience as a software developer.\n  - Experience as a Full Stack Developer and Team Lead, demonstrating leadership skills.\n  - Experience with modern technologies such as Python, AWS, Docker, and Scala.\n  - From Colombia, which is one of the approved countries.\n  - Experience as a tech lead is not explicitly mentioned.\n  - Experience with Java may indicate a focus on older technologies.\n  - Recent experience as a manager may suggest a shift away from individual contributor roles.\n  - Experience with C and Java may indicate a lack of experience with modern tools and frameworks.\n",
  "level": "IC4",
  "score_experience": 72,
  "score_tech_lead": true,
  "score_modern_tech": "Some",
  "score_country": true
}
```
"""

human_template = """\
Analyze the following candidate:

```
{candidate}
```
"""


In [52]:

# Set up desired response structure to be returned by the LLM
from pydantic.main import BaseModel, Field
from langchain.output_parsers import OutputFixingParser, PydanticOutputParser

class CandidateCurationSummaryModel(BaseModel):
    pass_: str = Field(
        description="Based on your analysis, should we proceed this candidate? Answer Yes if yes, No if not, or Needs More Information if you do not have enough information."
    )
    rejection_reason: str = Field(
        description="If you decided not to proceed with this candidate, share the code for your rejection. Otherwise, respond 'N/A'."
    )
    summary: str = Field(
        description="In under 150 words, list the good and bad things about this candidate in bullet points. This should capture your thoughts on why we should proceed with or reject this candidate."
    )
    level: str = Field(
        description="Based on your analysis, what do you determie the candidate's software engineering level on the Individual Contributor scale (from IC1 to IC6, with IC1 being Intern-level, IC2 Junior-level, IC3 Career-level, IC4 Advanced-level, IC5 Expert-level, and IC6 being Principal-level)? Provide an answer between IC1 and IC6, or if you did not have an answer respond 'N/A'."
    )
    score_experience: int = Field(
        description="How many months of work experience as a software developer does this candidate have? Make sure to include their current work up to today's date, which is August 3, 2023. Answer in an integer. If you think the answer is 'N/A', return 0 instead."
    )
    score_tech_lead: bool = Field(
        description="Does this candidate have any experience as a tech/team lead? Answer True or False. If this information is not available, answer False."
    )
    score_modern_tech: str = Field(
        description="Does this candidate have experience working with any of the following skills: [Ruby, Django, AWS, Typescript, Docker, Node, React, JQuery, Python, Angular, .NET, Java]? Answer Many if they have 3 or more, Some with 1-3 of these skills, and None if they have 0 of these skills. Answer None if you are unsure."
    )
    score_country: bool = Field(
        description="Is this candidate from one of the valid countries we hire from? Answer True or False."
    )

parser = PydanticOutputParser(pydantic_object=CandidateCurationSummaryModel)
fix_format_parser = OutputFixingParser.from_llm(parser=parser, llm=chat)

# Format the response instructions to pass into the templates
format_instructions = fix_format_parser.get_format_instructions()

In [41]:
# Generate the formatted chat conversation to pass to the LLM using the templates and response structure

# Generate the messages from templates
system_prompt = PromptTemplate(
    template=system_template,
    input_variables=[],
    partial_variables={"format_instructions": format_instructions},
)
system_message = SystemMessagePromptTemplate(prompt=system_prompt)


human_prompt = PromptTemplate(
    template=human_template,
    input_variables=["candidate"]
)
human_message = HumanMessagePromptTemplate(prompt=human_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])


### Run the test for many *candidates*

In [40]:
def get_candidate_summary(candidate):
  # Run the test
  messages = chat_prompt.format_prompt(candidate=candidate, format_instructions=format_instructions).to_messages()
  # Request the response and print API usage
  with get_openai_callback() as cb:
      response = chat(messages)
      return response, cb

In [ ]:
# Set up a list of candidates candidates
# Candidates for this test should have work experience, skills, and education
# Drop in the query you use here so we can reference it and modify it
def read_data_from_json_file(file_name):
  import json
  file_path = f"/content/sample_data/{file_name}"

  try:
      with open(file_path, 'r') as file:
          data = json.load(file)
          return data
  except FileNotFoundError:
      print("File not found or path is incorrect.")
  except json.JSONDecodeError:
      print("Error decoding JSON data.")

In [ ]:
# Create a candidate score using inferred data about the candidate
def generate_score(candidate_res: dict) -> float:
    try:
        value = 5
        ai_level = candidate_res["ai_level"]
        ai_score_experience = candidate_res["ai_score_experience"]
        ai_score_tech_lead = candidate_res["ai_score_tech_lead"]
        ai_score_modern_tech = candidate_res["ai_score_modern_tech"]
        ai_score_country = candidate_res["ai_score_country"]

        if ai_level == "IC1":
            value *= 0.1
        elif ai_level == "IC2":
            value *= 0.5
        elif ai_level == "IC4":
            value *= 1.3
        elif ai_level == "IC5":
            value *= 2
        elif ai_level == "IC6":
            value *= 2.5

        value *= max(0, min(2.5, ai_score_experience / 24))

        if ai_score_tech_lead:
            value *= 2

        if ai_score_modern_tech == "Many":
            value *= 2
        elif ai_score_modern_tech == "Some":
            value *= 1.5
        elif ai_score_modern_tech == "None":
            value *= 0.8

        if not ai_score_country:
            value *= 0.1

        return round(value, 1)
    except KeyError:
        print("Error: Missing or unexpected value in candidate_res dictionary.")
        return None

In [ ]:
# Filter candidates based on data - we are only interested in candidates that
# have education, skills and work history
def process_candidates(candidates):
  total_tokens = 0
  total_cost = 0
  total_passes = 0
  total_passing_score = 0
  total_fails = 0
  total_failing_score = 0
  total_unknowns = 0
  candidate_results = []
  for candidate in candidates:
    candidate_res = {"candidate_id": candidate["id"]}
    rejected_reasons = [r["rejected_reason"] for r in candidate["candidate_curation_workflows"] if r["rejected_reason"] != None]
    candidate_res["rejected_reasons"] = rejected_reasons

    # Make sure we're not sending candidate_curation_workflows to AI
    sanitized_candidate = {key: value for key, value in candidate.items() if key != "candidate_curation_workflows"}
    res, cb = get_candidate_summary(sanitized_candidate)
    print(res)

    # Notice we're using fix_format parser to get exact output defined above
    dict_res = fix_format_parser.parse(res.content).dict()

    ai_pass = dict_res["pass_"]
    candidate_res["ai_pass"] = ai_pass
    candidate_res["ai_rejection_reason"] = dict_res["rejection_reason"]
    candidate_res["ai_level"] = dict_res["level"]
    candidate_res["ai_score_experience"] = dict_res["score_experience"]
    candidate_res["ai_score_tech_lead"] = dict_res["score_tech_lead"]
    candidate_res["ai_score_modern_tech"] = dict_res["score_modern_tech"]
    candidate_res["ai_score_country"] = dict_res["score_country"]
    candidate_res["ai_summary"] = dict_res["summary"]

    ai_score = generate_score(candidate_res)
    candidate_res["ai_score"] = ai_score

    candidate_results.append(candidate_res)
    total_tokens += cb.total_tokens
    total_cost += cb.total_cost
    if ai_pass == "Yes":
      total_passes+=1
    elif ai_pass == "No":
      total_fails+=1
    else:
      total_unknowns+=1

    if ai_score >= 3:
        total_passing_score+= 1
    elif ai_score <3:
        total_failing_score+= 1

  return {
      "candidate_results": candidate_results,
      "total_passes": total_passes,
      "total_fails": total_fails,
      "total_passing_score": total_passing_score,
      "total_failing_score": total_failing_score,
      "total_unknowns": total_unknowns,
      "total_cost": total_cost,
      "total_tokens": total_tokens
  }

In [ ]:
def print_results(cand_results, total_candidates, is_pass):
  total_passes = cand_results["total_passes"]
  total_fails = cand_results["total_fails"]
  print(f"Total AI Passes: {total_passes}")
  print(f"Total AI Fails: {total_fails}")
  print(f"Total AI Unknowns: {cand_results['total_unknowns']}")
  total_correct = total_passes if is_pass else total_fails
  print(f"Percentage AI accuracy: {total_correct/total_candidates * 100} %")

  print("\nTest Results - Scored Approach")
  total_passing_score = cand_results["total_passing_score"]
  total_failing_score = cand_results["total_failing_score"]
  print(f"Total AI Passing Scores: {total_passing_score}")
  print(f"Total AI Failing Scores: {total_failing_score}")
  print(f"Total AI Unknowns: {cand_results['total_unknowns']}")
  total_correct_combined = total_passing_score if is_pass else total_failing_score
  print(f"Percentage AI accuracy combined: {total_correct_combined/total_candidates * 100} %")

  print("\nTest API Consumption")
  print(f"Number of tokens used for the test: {cand_results['total_tokens']} ")
  print(f"Cost for the test: {cand_results['total_cost']} USD")
  print("\nRaw Data")
  print("\n".join(str(generate_score(item)) + " " + str(item) for item in cand_results["candidate_results"]))

  df = pd.DataFrame(cand_results["candidate_results"])
  display(df)

# Run the test here

In [ ]:
# DON'T RUN THIS BLOCK
# IT IS JUST A REFERENCE TO THE GRAPHQL QUERY TO GENERATE CANDIDATE LISTS

# Passed Candidates Query. Simply run this in hasura console and copy paste the results
# in the file: sample_data/passed_candidates.json
query GetPassedCandidates {
  candidate(
    limit: 20
    order_by: {created_at: desc}
    where: {
    candidate_work_experiences: {id: {_is_null: false}}
    candidate_educations: {id: {_is_null: false}}
    candidate_skills: {id: {_is_null: false}}
     furthest_candidate_curation_workflow: {status: {_in: [
          ACCEPTED,
          PHONE_SCREEN_PASSED,
          PHONE_SCREEN_PASSED,
          PHONE_SCREEN_NO_SHOW
          PHONE_SCREEN_COMPLETED
          PHONE_SCREEN_SCHEDULED
          PHONE_SCREEN_SCHEDULING
        ]}
      }
    }

  ) {
    id
    country
    candidate_skills {
      skill {
        name
      }
    }
    candidate_work_experiences {
      job_title
      start_date
      end_date
    }
    candidate_educations {
      degree
      school_name
      start_year
      graduation_year
    }
    candidate_curation_workflows {
      rejected_reason
    }
  }
}


# Failed Candidates Query. Simply run this in hasura console and copy paste the results
# in the file: sample_data/failed_candidates.json
query GetFailedCandidates {
  candidate(
    limit: 20
    order_by: {created_at: desc}
    where: {
    candidate_work_experiences: {id: {_is_null: false}}
    candidate_educations: {id: {_is_null: false}}
    candidate_skills: {id: {_is_null: false}}
      candidate_curation_workflows: {rejected_reason: {_nin: [WORK_AUTHORIZATION, VALUABLE_PROFILE, TIMING]}}
     furthest_candidate_curation_workflow: {status: {_nin: [
          ACCEPTED,
          PHONE_SCREEN_PASSED,
          PHONE_SCREEN_PASSED,
          PHONE_SCREEN_NO_SHOW,
          PHONE_SCREEN_COMPLETED,
          PHONE_SCREEN_SCHEDULED,
          PHONE_SCREEN_SCHEDULING,
      		NEW_NEEDS_REVIEW
        ]}
      }
    }

  ) {
    id
    country
    candidate_skills {
      skill {
        name
      }
    }
    candidate_work_experiences {
      job_title
      start_date
      end_date
    }
    candidate_educations {
      degree
      school_name
      start_year
      graduation_year
    }
    candidate_curation_workflows {
      rejected_reason
    }
  }
}

In [47]:
print("Running test for 'passed' candidates...")
p_cands = read_data_from_json_file("passed_candidates.json")["data"]["candidate"]
total_p_cands = len(p_cands)
p_cand_results = process_candidates(p_cands)
print_results(p_cand_results, total_p_cands, True)

Running test for 'passed' candidates...
content='1. The candidate has a total of 13 years of experience as a software developer. Their career progression has been as follows:\n   - Founding Product Engineer (2020-present)\n   - Staff Engineer (2019-2020)\n   - Senior Engineer (2016-2019)\n   - Developer (2010-2016)\n\n   The candidate has leveled up relatively quickly, starting as a developer and progressing to senior and staff engineer roles. They have primarily worked as a developer throughout their career.\n\n2. Based on the criteria provided, we should proceed with this candidate in the hiring process.\n\n{\n  "pass_": "Yes",\n  "rejection_reason": "N/A",\n  "summary": "Good things about this candidate:\\n- Has 13 years of experience as a software developer\\n- Has progressed quickly in their career\\n- Has experience in frontend development using React and Typescript\\n- Has experience with modern tools and frameworks\\n\\nLevel: IC4",\n  "level": "IC4",\n  "score_experience": 156

,candidate_id,rejected_reasons,ai_pass,ai_rejection_reason,ai_level,ai_score_experience,ai_score_tech_lead,ai_score_modern_tech,ai_score_country,ai_summary,ai_score
0,650789,[],Yes,N/A,IC4,156,False,Many,True,Good things about this candidate:\n- Has 13 ye...,32.5
1,650124,[],Yes,N/A,IC2,31,False,Many,True,- Good: The candidate has relevant work experi...,6.5
2,649626,[],Yes,N/A,IC3,48,False,Many,True,- The candidate has 4 years of experience as a...,20.0
3,648288,[],Yes,N/A,IC4,108,False,Many,True,- The candidate has 9 years of work experience...,32.5
4,648034,[],Yes,N/A,IC3,24,False,Many,True,- Good: Two years of work experience as a soft...,10.0
5,647493,[],Yes,N/A,N/A,0,False,None,True,- Good: The candidate has relevant work experi...,0.0
6,647124,[],Yes,N/A,IC3,48,False,Many,True,- The candidate has 4 years of experience as a...,20.0
7,645953,[],Yes,N/A,N/A,228,False,Some,True,- Good: The candidate has 19 years of work exp...,18.8
8,644944,[],Yes,N/A,IC4,96,False,Some,True,- Good: 8 years of software development experi...,24.4
9,644860,[],Yes,N/A,IC3,57,False,Many,True,Good things about this candidate:\n- Has 4+ ye...,23.8


In [53]:
print("\n\nRunning test for 'failed' candidates...")
f_cands = read_data_from_json_file("failed_candidates.json")["data"]["candidate"]
total_f_cands = len(f_cands)
f_cand_results = process_candidates(f_cands)
print_results(f_cand_results, total_f_cands, False)



Running test for 'failed' candidates...
content='1. The candidate has a total of 3 years and 8 months of work experience as a software developer. They started as a Purchasing Engineer in July 2019 and transitioned to a Project Engineer role in January 2020. Their most recent work experience is directly related to software development.\n\n2. The candidate is from Chile, which is one of the approved countries.\n\n3. The candidate has 3 years and 8 months of total hands-on software development experience, which exceeds the minimum requirement of 2 years.\n\n4. There is no information provided about the candidate\'s experience as a tech/team lead.\n\n5. There is no information provided about the candidate\'s role as an individual contributor or manager.\n\n6. The candidate has experience with modern tools and frameworks such as React, Postgres, Node, Typescript, Redux, REST API, Git, MongoDB, HTML, and CSS.\n\nBased on the evaluation, we should proceed with this candidate in the hiring p

,candidate_id,rejected_reasons,ai_pass,ai_rejection_reason,ai_level,ai_score_experience,ai_score_tech_lead,ai_score_modern_tech,ai_score_country,ai_summary,ai_score
0,651220,[NOT_CORE_ROLE],Yes,N/A,N/A,44,False,Many,True,The candidate has 3 years and 8 months of work...,18.3
1,651212,[TOO_JUNIOR],Yes,N/A,IC3,31,False,Many,True,"- Good experience with React, JavaScript, PHP,...",12.9
2,651210,[NOT_CORE_ROLE],No,NOT_CORE_ROLE,N/A,0,False,Some,True,- The candidate does not have work experience ...,0.0
3,651200,[OUTSIDE_GEO],No,"NOT_CORE_ROLE, OUTSIDE_GEO",N/A,96,False,Some,False,Good: The candidate has 8 years of experience ...,1.9
4,651027,[TOO_JUNIOR],Yes,N/A,N/A,84,False,Many,True,Good things about this candidate:\n- 7 years o...,25.0
5,650999,[NOT_CORE_FOCUS],Yes,N/A,IC3,144,False,Many,True,- Good: The candidate has 12 years of experien...,25.0
6,650964,[OUTSIDE_GEO],No,NOT_CORE_ROLE,N/A,29,False,Some,False,The candidate has a total of 2 years and 5 mon...,0.9
7,650845,[NOT_CORE_ROLE],No,NOT_CORE_ROLE,N/A,33,False,Some,True,Limited work experience as a software develope...,10.3
8,650809,[NOT_CORE_ROLE],No,NOT_CORE_ROLE,N/A,24,False,None,True,- The candidate has 2 years of work experience...,4.0
9,650798,[NOT_CORE_FOCUS],Yes,N/A,N/A,24,False,Some,True,- The candidate has 2 years of experience as a...,7.5
